In [ ]:
!pip install pymongo
import pymongo
cli = pymongo.MongoClient(host='34.210.97.79')

In [2]:
cli.database_names()

['admin', 'local', 'wiki_mongo_database', 'wiki_mongo_db']

In [3]:
import pandas as pd
import numpy as np
from IPython import display
from bs4 import BeautifulSoup
import requests
!pip install nltk
import nltk

import re
from spacy.en import STOP_WORDS
from spacy.en import English
nlp = English()

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
wiki_mongo_database = cli.wiki_mongo_database
wiki_mongo_collection = cli.wiki_mongo_database.wiki_mongo_collection
wiki_mongo_collection.count()

5665

In [ ]:
df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_mongo_collection.find({})))
df.sort(columns='clean_content',ascending=True)

In [ ]:
def cleaner(text):
    text = text.lower()
    text = re.sub('<.{0,3}>',' ',text)
    text = re.sub('{.*\.*}',' ',text)
    text = re.sub('{.*\+.*}',' ',text)
    #text = re.sub('[\W]',' ',text)
    text = re.sub('[^a-z]',' ',text)
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    #text = re.sub('/(?<!\S).(?!\S)\s*/',' ',text)
    text = re.sub('aa','a',text)
    text = re.sub('[0-9]','',text)
    text = re.sub('\s+',' ',text)
    text = ' '.join([i.lemma_ for i in nlp(text) 
                     if i.orth_ not in STOP_WORDS])
    return text

In [ ]:
for i in range(wiki_mongo_collection.count()):
    cur = wiki_mongo_collection.find()[i]
    content = cur['content']
    cleaned_content = cleaner(content)
    wiki_mongo_collection.update_one(
        {'content':content},
        {'$set': {'clean_content':cleaner(content)}}
        )

In [ ]:
wiki_mongo_collection.delete_many({'title':{'$regex':"File:.*"}})
wiki_mongo_collection.delete_many({"clean_content":''})
wiki_mongo_collection.delete_many( { 'clean_content': None })

In [5]:
wiki_mongo_collection.count()

5665

In [ ]:
df[df['clean_content'].isnull() == True]

In [ ]:
df = pd.DataFrame(list(cli.wiki_mongo_database.wiki_mongo_collection.find({})))
print(df.count())
df.sort('clean_content').head(10)